# 3.网页排名
## 3.1 网页点击率排名表

In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/wangye?charset=utf8')
sql = pd.read_sql('all_gzdata', engine, chunksize = 10000)
# g = [pd.merge(f,i[['fullURLId','fullURL','realIP']],right_on = 'realIP',left_index=True,how ='left') for i in sql]

def clickfreq(i): #自定义统计函数
    j = i[['fullURL','fullURLId','realIP']][i['fullURL'].str.contains('\.html')]
    return j

counts1 = [clickfreq(i) for i in sql] # 分块统计各个IP的出现次数
counts1 = pd.concat(counts1)
 
counts1_ = counts1['fullURL'].value_counts()
counts1_ = pd.DataFrame(counts1_)

counts1_.columns = [u'点击次数']
counts1_.index.name = u'网址'
a = counts1_.sort_values(u'点击次数',ascending=False).iloc[:20,:]
a

D:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 478")
  result = self._query(query)


,点击次数
网址,
http://www.lawtime.cn/faguizt/23.html,944
http://www.lawtime.cn/info/shuifa/slb/2012111978933.html,643
http://www.lawtime.cn/faguizt/43.html,516
http://www.lawtime.cn/ask/browse_a1401.html,507
http://www.lawtime.cn/faguizt/9.html,505
http://www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693.html,484
http://www.lawtime.cn/ask/exp/13655.html,379
http://www.lawtime.cn/info/shuifa/slb/2012111978933_2.html,378
http://www.lawtime.cn/faguizt/41.html,369


## 3.2 点击次数大于50

In [2]:
b = counts1_.reset_index()
c = b[b[u'点击次数']>50][b[u'网址'].str.contains('/\d+?_*\d+?\.html')]
c.set_index(u'网址',inplace=True)
c.sort_index(inplace = True)
# savetosql(c, 'count355')# 并保存到数据库中
c

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,点击次数
网址,
http://www.lawtime.cn/ask/exp/13445.html,129
http://www.lawtime.cn/ask/exp/13653.html,57
http://www.lawtime.cn/ask/exp/13655.html,379
http://www.lawtime.cn/ask/exp/17357.html,86
http://www.lawtime.cn/ask/exp/8495.html,240
http://www.lawtime.cn/ask/online/138.html,245
http://www.lawtime.cn/ask/online/139.html,301
http://www.lawtime.cn/faguizt/11.html,210
http://www.lawtime.cn/faguizt/117.html,356


## 3.3 翻页网页统计，对浏览网页翻页的情况进行统计

In [3]:
# 获取网址中以http://与.html中间的主体部分,即去掉翻页的内容，即去掉尾部"_d"
import re
import numpy as np
pattern = re.compile('http://(.*\d+?)_\w+_\w+\.html$|http://(.*\d+?)_\w+\.html$|http://(.*\w+?).html$',re.S)
c['websitemain'] = np.nan
for i in range(len(c)):
    items = re.findall(pattern, c.index[i])
    if len(items)== 0:
        temp = np.nan
    else:
        for j in items[0]:
            if j !='':
                temp = j
    c.iloc[i,1] = temp
c

,点击次数,websitemain
网址,,
http://www.lawtime.cn/ask/exp/13445.html,129,www.lawtime.cn/ask/exp/13445
http://www.lawtime.cn/ask/exp/13653.html,57,www.lawtime.cn/ask/exp/13653
http://www.lawtime.cn/ask/exp/13655.html,379,www.lawtime.cn/ask/exp/13655
http://www.lawtime.cn/ask/exp/17357.html,86,www.lawtime.cn/ask/exp/17357
http://www.lawtime.cn/ask/exp/8495.html,240,www.lawtime.cn/ask/exp/8495
http://www.lawtime.cn/ask/online/138.html,245,www.lawtime.cn/ask/online/138
http://www.lawtime.cn/ask/online/139.html,301,www.lawtime.cn/ask/online/139
http://www.lawtime.cn/faguizt/11.html,210,www.lawtime.cn/faguizt/11
http://www.lawtime.cn/faguizt/117.html,356,www.lawtime.cn/faguizt/117


In [4]:
# 获取所有网页主体的网页数
d = c['websitemain'].value_counts()
d = pd.DataFrame(d)
d

,websitemain
www.lawtime.cn/info/hetong/ldht/201311152872128,2
www.lawtime.cn/info/minshi/fagui/2013051382463,2
www.lawtime.cn/info/shuifa/slb/2012111978933,2
www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693,2
www.lawtime.cn/ask/exp/17357,1
www.lawtime.cn/faguizt/27,1
www.lawtime.cn/info/laodong/gongshixiujia/201412253312096,1
www.lawtime.cn/faguizt/131,1
www.lawtime.cn/faguizt/168,1
www.lawtime.cn/faguizt/25,1


In [5]:
# 统计网页主体出现次数为不少于二次的，即存在翻页的网址
e = d[d['websitemain']>=2]
e.columns=['Times']#记录某网页及子网页出现的此处
e.index.name='websitemain'# 主网页
 
e['num'] = np.arange(1,len(e)+1) 
f = pd.merge(c,e,left_on='websitemain',right_index=True,how='right')
f.sort_index(inplace=True)
f['per'] = np.nan
f# 相同num的网页是拥有同一网页主体

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,点击次数,websitemain,Times,num,per
网址,,,,,
http://www.lawtime.cn/info/hetong/ldht/201311152872128_2.html,95,www.lawtime.cn/info/hetong/ldht/201311152872128,2,1,NaN
http://www.lawtime.cn/info/hetong/ldht/201311152872128_3.html,62,www.lawtime.cn/info/hetong/ldht/201311152872128,2,1,NaN
http://www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693.html,484,www.lawtime.cn/info/hunyin/lhlawlhxy/201107071...,2,4,NaN
http://www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693_2.html,298,www.lawtime.cn/info/hunyin/lhlawlhxy/201107071...,2,4,NaN
http://www.lawtime.cn/info/minshi/fagui/2013051382463_4.html,175,www.lawtime.cn/info/minshi/fagui/2013051382463,2,2,NaN
http://www.lawtime.cn/info/minshi/fagui/2013051382463_5.html,53,www.lawtime.cn/info/minshi/fagui/2013051382463,2,2,NaN
http://www.lawtime.cn/info/shuifa/slb/2012111978933.html,643,www.lawtime.cn/info/shuifa/slb/2012111978933,2,3,NaN
http://www.lawtime.cn/info/shuifa/slb/2012111978933_2.html,378,www.lawtime.cn/info/shuifa/slb/2012111978933,2,3,NaN


In [6]:
# 统计翻子页的点击率与上一页网页点击率的比重（注意：用此处这个方法对网页翻页后序号有10页及以上的合适
def getper(x):
    x.sort_index(inplace=True) #必须先排序将网页
    x
    for i in range(len(x)-1):
        x.iloc[i+1,-1] = x.iloc[i+1,0]/x.iloc[i,0]
    return x    
        
 
result = pd.DataFrame([]) # 用一个空表格记录值
for i in range(1,f['num'].max()+1):#count36['num'].max()+1
    k= getper(f[f['num'] == i])
    result = pd.concat([result,k])# 每次进行一次操作时

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
f['Times'].value_counts() # 由统计结果看，只有一个主网址出现过10次及以上，该数据采用上述方法会出问题，因此，在结果中将其剔除后观察剩余数据

2    8
Name: Times, dtype: int64

In [10]:
flipPageResult = result[result['Times']<10]
#保存的表名命名格式为“1_3_k此表功能名称”，是此小节生成的第1张表格，功能为flipPageResult：统计翻子页的点击率与上一页网页点击率的比重
flipPageResult.to_excel('./tmp/3_flipPageResult.xlsx')
flipPageResult

,点击次数,websitemain,Times,num,per
网址,,,,,
http://www.lawtime.cn/info/hetong/ldht/201311152872128_2.html,95,www.lawtime.cn/info/hetong/ldht/201311152872128,2,1,NaN
http://www.lawtime.cn/info/hetong/ldht/201311152872128_3.html,62,www.lawtime.cn/info/hetong/ldht/201311152872128,2,1,0.652632
http://www.lawtime.cn/info/minshi/fagui/2013051382463_4.html,175,www.lawtime.cn/info/minshi/fagui/2013051382463,2,2,NaN
http://www.lawtime.cn/info/minshi/fagui/2013051382463_5.html,53,www.lawtime.cn/info/minshi/fagui/2013051382463,2,2,0.302857
http://www.lawtime.cn/info/shuifa/slb/2012111978933.html,643,www.lawtime.cn/info/shuifa/slb/2012111978933,2,3,NaN
http://www.lawtime.cn/info/shuifa/slb/2012111978933_2.html,378,www.lawtime.cn/info/shuifa/slb/2012111978933,2,3,0.587869
http://www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693.html,484,www.lawtime.cn/info/hunyin/lhlawlhxy/201107071...,2,4,NaN
http://www.lawtime.cn/info/hunyin/lhlawlhxy/20110707137693_2.html,298,www.lawtime.cn/info/hunyin/lhlawlhxy/201107071...,2,4,0.615702
